In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn  
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, make_scorer
from keras import preprocessing as pre
from keras import Sequential
from keras import layers as lay
from keras import layers, metrics, optimizers, models, losses, utils
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np  
from PIL import Image
import glob
import tensorflow as tf



In [2]:
sklearn.set_config(transform_output="pandas")

In [3]:

dir = r"C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Data\LEGO brick images v1"

In [4]:
df,df_val= pre.image_dataset_from_directory(
    directory = dir,
    labels='inferred', 
    label_mode='categorical',
    color_mode="grayscale",
    image_size=(200, 200),
    batch_size = 128,
    shuffle=True,
    seed=42,
    interpolation="bilinear",
    validation_split=0.2,
    subset='both'
)

Found 6379 files belonging to 16 classes.
Using 5104 files for training.
Using 1275 files for validation.


In [5]:
model = Sequential(
       layers=(
           
           lay.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(200, 200, 1)),
           lay.MaxPooling2D(2,2),

           lay.Dense(64,activation='relu'),
           lay.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),

           lay.Dropout(0.2),
           lay.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),
           
           lay.Dense(32, activation='relu'),
           lay.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
           lay.MaxPooling2D(2,2),
           
           lay.Flatten(),
           lay.Dense(16, activation='softmax')
           
           )
       )



c:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy'])


In [7]:
epoche = 10

In [8]:

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 99, 99, 64)     │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23, 23, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 21, 21, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │       204,816 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 282,096 (1.08 MB)

 Trainable params: 282,096 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(df,epochs=epoche,validation_data=df_val,batch_size=32)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 177s 4s/step - accuracy: 0.2544 - loss: 4.1977 - val_accuracy: 0.6047 - val_loss: 1.1110
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.6683 - loss: 0.8869 - val_accuracy: 0.7482 - val_loss: 0.6595
Epoch 3/10
 7/40 ━━━━━━━━━━━━━━━━━━━━ 1:56 4s/step - accuracy: 0.7974 - loss: 0.5553

In [ ]:
predictions =  model.predict(df)

In [ ]:
print(predictions)

In [ ]:
model.export(filepath=r'C:\Users\bruno\OneDrive\Documenti\Deep_learning_project\Testing')